In [15]:
import requests
import pandas as pd
import numpy as np
import math
from bs4 import BeautifulSoup
from datetime import datetime

In [28]:
url= 'https://www2.hm.com/en_us/men/products/jeans.html'

headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36 OPR/37.0.2178.54'}

page = requests.get( url, headers=headers )


soup= BeautifulSoup(page.text,'html.parser')


#coletando o total de itens de todas as páginas
total_item= soup.find_all('h2',class_='load-more-heading')[0].get('data-total')

#calculando o número de paginas que preciso calcular para pesquisar pelas vitrines da loja
page_number =math.ceil(int(total_item)/36)

page_number

#montando url para pegar todos os produtos da página
url2= url +'?page-size=' + str(int(page_number *36))
url2

'https://www2.hm.com/en_us/men/products/jeans.html?page-size=108'

In [33]:
#url= 'https://www2.hm.com/en_us/productpage.0985159005.html'
url = 'https://www2.hm.com/en_us/men/products/jeans.html?page-size='+ str(url2)

headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36 OPR/37.0.2178.54'}

page = requests.get( url, headers=headers )

soup= BeautifulSoup(page.text,'html.parser')

In [36]:
# Just One Products

#Beautifull Souo object
#soup= BeautifulSoup(page.text,'html.parser')

### ===================== color name ======================

products_list= soup.find_all('a',class_='filter-option miniature')

#product id
products_id = [p.get('data-articlecode') for p in products_list]

#color name
color_name = [p.get('data-color') for p in products_list]


#create data frame

df_color = pd.DataFrame([products_id,color_name]).T

#rename the columns of DF
df_color.columns= ['product_id','color_name']

#create new colum styly_id
df_color['stlye_id'] = df_color['product_id'].apply(lambda x: x[:-3])

#create new colum color_id
df_color['color_id'] = df_color['product_id'].apply(lambda x: x[-3:])

# ============================== composition =============================== #

#composition
composition_list= soup.find_all('div', class_ = 'details-attributes-list-item')

product_composition = [list(filter (None, p.get_text().split('\n') ) ) for p in composition_list]

if product_composition != []:
    
    #created the dataframe aux
    df_aux= pd.DataFrame(product_composition).T

    #renome the columns
    df_aux.columns= df_aux.iloc[0]

    #select columns 
    df_aux= df_aux[['Fit','Composition','Art. No.']]

    #delete the first row and delete the columns that is the empty
    df_aux= df_aux.iloc[1:3]

    #copying the code and the model to row the below
    df_aux = df_aux.fillna(method='ffill')

    #rename the data frame to a new variable
    df_composition = df_aux.copy(True)

    #create new colum styly_id
    df_composition['stlye_id'] =df_composition['Art. No.'].apply(lambda x: x[:-3])

    #create new colum color_id
    df_composition['color_id'] = df_composition['Art. No.'].apply(lambda x: x[-3:])

    #merge dataframe of color and composition 
    data_sku= pd.merge(df_color,df_composition[['stlye_id','Fit','Composition']], how='left', on ='stlye_id')

In [37]:
data_sku

,product_id,color_name,stlye_id,color_id,Fit,Composition
0,0985159001,Black,0985159,001,Skinny fit,"Shell: Cotton 99%, Spandex 1%"
1,0985159001,Black,0985159,001,Skinny fit,Pocket lining: Cotton 100%
2,0985159002,Denim blue,0985159,002,Skinny fit,"Shell: Cotton 99%, Spandex 1%"
3,0985159002,Denim blue,0985159,002,Skinny fit,Pocket lining: Cotton 100%
4,0985159003,Dark gray,0985159,003,Skinny fit,"Shell: Cotton 99%, Spandex 1%"
5,0985159003,Dark gray,0985159,003,Skinny fit,Pocket lining: Cotton 100%
6,0985159004,Light denim blue,0985159,004,Skinny fit,"Shell: Cotton 99%, Spandex 1%"
7,0985159004,Light denim blue,0985159,004,Skinny fit,Pocket lining: Cotton 100%
8,0985159006,Dark denim blue,0985159,006,Skinny fit,"Shell: Cotton 99%, Spandex 1%"
9,0985159006,Dark denim blue,0985159,006,Skinny fit,Pocket lining: Cotton 100%


In [5]:
df_aux.iloc[0]

0
Fit                               Skinny fit
Composition    Shell: Cotton 99%, Spandex 1%
Art. No.                          0985159005
Name: 1, dtype: object